In [5]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add, uint
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

@si.coroutine(inputs={"I" : si.Array(4, si.Bits(16))})
def Serializer4():
    data = [bits(0, 16) for _ in range(3)]
    I = yield
    while True:
        O = I[0]
        for i in range(3):
            data[i] = I[i + 1]
        I = yield O
        for i in range(3):
            O = data[i]
            I = yield O

        
serializer4 = si.compile(Serializer4(), file_name="build/silica_serializer4.py")
with open("build/silica_serializer4.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator

Serializer4 = DefineCircuit("Serializer4", "O", Out(Bits(16)), "I", In(Array(4,Bits(16))), *ClockInterface(has_ce=False))

Buffer = DefineCircuit("__silica_BufferSerializer4", "I", In(Bits(5)), "O", Out(Bits(5)))
wire(Buffer.I, Buffer.O)
EndDefine()
__silica_path_state = Buffer()
__silica_yield_state = Register(5, init=1, has_ce=False)

wireclock(Serializer4, __silica_yield_state)

__silica_yield_state_next = Or(5, 5)
wire(__silica_yield_state_next.O, __silica_yield_state.I)
I = Serializer4.I
__silica_yield_state_next_0 = And(2, 5)
wire(__silica_yield_state_next_0.O, __silica_yield_state_next.I0)
for __silica_j in range(5):
    wire(__silica_yield_state_next_0.I0[__silica_j], __silica_path_state.O[0])
__silica_yield_state_next_1 = And(2, 5)
wire(__silica_yield_state_next_1.O, __silica_yield_state_next.I1)
for __silica_j in range(5):
    wire(__silica_yi

In [6]:
!magma -o coreir -m coreir -t Serializer4 build/silica_serializer4.py
!coreir -i build/silica_serializer4.json -o build/silica_serializer4.v
!yosys -p 'synth_ice40 -top Serializer4 -blif build/silica_serializer4.blif' build/silica_serializer4.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_serializer4.txt build/silica_serializer4.blif
!icetime -tmd hx1k build/silica_serializer4.txt  | grep -B 2 "Total path delay"

In Run Generators

Modified?: Yes
2.27. Printing statistics.

=== Serializer4 ===

   Number of wires:               1353
   Number of wire bits:           4800
   Number of public wires:        1319
   Number of public wire bits:    4766
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                151
     SB_DFF                         53
     SB_LUT4                        98

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_serializer4.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          81 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          103 / 1280
  DFF        53
  CARRY      0
  CARRY, DFF 0
  DFF PASS   5
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promoted 

In [9]:
!yosys -p 'synth_ice40 -top serializer -blif build/verilog_serializer.blif' ../verilog/serializer.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/verilog_serializer.txt build/verilog_serializer.blif
!icetime -tmd hx1k build/verilog_serializer.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== serializer ===

   Number of wires:                 27
   Number of wire bits:            151
   Number of public wires:           7
   Number of public wire bits:     131
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 86
     SB_DFFE                        48
     SB_DFFSR                        2
     SB_LUT4                        36
seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/verilog_serializer.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          81 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          84 / 1280
  DFF        50
  CARRY      0
  CARRY, DFF 0
  DFF PASS   48
  CARRY PASS 0
BRAMs        0 / 16
WARMBOOTs    0 / 1
PLLs         0 / 1

place_constraints...
promote_globals...
  promot